# Mean vorticity flux divergence and vorticity tendency

In [1]:
import os
import sys
sys.path.append('../..')
import glob
import time
import xarray
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean
from numba import float64, guvectorize
import cimf as c
from ddxddy import ddxND, ddyND
%load_ext autoreload
%autoreload 2

plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['font.cursive'] = ['Helvetica-Oblique']
plt.rcParams['pdf.fonttype'] = 42

In [2]:
p_ref = 100000
cp = 1004
Rdry = 286.9
kappa = Rdry/cp

In [3]:
dataloc = "/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989interped/"
#dataloc2 = "/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989/"
files = sorted(glob.glob(dataloc+'*.nc'))
#files2 = sorted(glob.glob(dataloc2+'*.nc'))

In [4]:
ds = xarray.open_mfdataset(files, concat_dim='valid_time', combine='nested')
ds = ds.swap_dims(theta='z')
ds = c.crop(ds)

11:46:49: crop()


In [5]:
ds['cimf'] = c.calc_cimf(ds)

11:47:55: calc_cimf()
11:47:55: masscontinuity(lagrange2d)
11:47:55: latlon2dxdy()
11:47:55: translational_velocity()
11:47:56: calc_dp(ds)
11:47:58: masstendency()
11:47:59: iwind2
11:48:01: FX, FY in masscontinuity()
11:48:06: ddx, ddy in masscontinuity()
11:48:23: integrate_dcimf()


In [6]:
ds = c.vorticity_flux(ds)

11:48:25: vorticity_flux()
11:48:25: Calculating dthetadt with (slightly) misaligned dp and sigma.
11:48:25: calc_dp(ds)
11:48:33: translational_velocity()
11:48:33: cartesian_gradient()
11:48:56: cartesian_gradient()
11:49:21: cartesian_gradient()
11:49:33: cartesian_gradient()
11:50:01: cartesian_gradient()
11:51:37: cartesian_gradient()
11:52:48: cartesian_gradient()
11:53:16: cartesian_gradient()
11:53:22: cartesian_gradient()
11:53:58: cartesian_gradient()
11:54:56: cartesian_gradient()
11:55:03: toPolar()
11:55:03: distance()


In [7]:
ds = c.vorticityflux_divergence(ds)

11:55:04: cartesian_gradient()
11:55:26: cartesian_gradient()
11:55:43: cartesian_gradient()
11:56:00: cartesian_gradient()
11:56:17: cartesian_gradient()


/Users/jasperdejong/opt/anaconda3/lib/python3.8/site-packages/numba/np/ufunc/gufunc.py:171: RuntimeWarning: divide by zero encountered in _ddthetagufunc
  return self.ufunc(*args, **kwargs)
/Users/jasperdejong/opt/anaconda3/lib/python3.8/site-packages/numba/np/ufunc/gufunc.py:171: RuntimeWarning: overflow encountered in _ddthetagufunc
  return self.ufunc(*args, **kwargs)


11:57:55: cartesian_gradient()


/Users/jasperdejong/opt/anaconda3/lib/python3.8/site-packages/numba/np/ufunc/gufunc.py:171: RuntimeWarning: divide by zero encountered in _ddthetagufunc
  return self.ufunc(*args, **kwargs)
/Users/jasperdejong/opt/anaconda3/lib/python3.8/site-packages/numba/np/ufunc/gufunc.py:171: RuntimeWarning: overflow encountered in _ddthetagufunc
  return self.ufunc(*args, **kwargs)


In [8]:
ds = c.convert_vorticityflux(ds)

11:59:49: convert_vorticityflux()
11:59:49: toPolar()
11:59:49: distance()
11:59:49: transform_vector()
11:59:49: transform_vector()
12:01:17: transform_vector()


In [9]:
dss = ds[['latitude','longitude','xc','yc','eta','Ja','Jh','Je','divJa','divJh','divJe','pres','cimf','sigma']]

In [10]:
dsm,_ = c.azimean(dss)

12:01:19: azimean()
12:01:19: toPolar()
12:01:19: distance()
12:01:19: convert_wind()
12:01:19: convert_wind(): no input, skipping
12:01:19: azimean_gufunc()
12:01:42: removing ['dy', 'dx'] from azimuthal mean dataset


/Users/jasperdejong/opt/anaconda3/lib/python3.8/site-packages/numba/np/ufunc/gufunc.py:171: RuntimeWarning: divide by zero encountered in _ddthetagufunc
  return self.ufunc(*args, **kwargs)
/Users/jasperdejong/opt/anaconda3/lib/python3.8/site-packages/numba/np/ufunc/gufunc.py:171: RuntimeWarning: overflow encountered in _ddthetagufunc
  return self.ufunc(*args, **kwargs)


In [16]:
dsm.to_netcdf("lg2_Jeta.nc")

In [19]:
dss

<xarray.Dataset>
Dimensions:     (valid_time: 37, dy: 301, dx: 301, z: 55)
Coordinates:
    time        datetime64[ns] 2017-09-05T12:00:00
  * valid_time  (valid_time) datetime64[ns] 2017-09-05T12:00:00 ... 2017-09-07
    step        (valid_time) timedelta64[ns] 00:00:00 ... 1 days 12:00:00
    theta       (z) float64 295.0 296.7 298.3 300.0 ... 500.0 600.0 700.0 800.0
  * dy          (dy) int64 -150 -149 -148 -147 -146 -145 ... 146 147 148 149 150
  * dx          (dx) int64 -150 -149 -148 -147 -146 -145 ... 146 147 148 149 150
Dimensions without coordinates: z
Data variables: (12/15)
    latitude    (valid_time, dy, dx) float32 12.16 12.16 12.16 ... 23.77 23.77
    longitude   (valid_time, dy, dx) float32 298.0 298.0 298.1 ... 298.8 298.8
    xc          (valid_time) int64 535 527 521 515 506 ... 300 293 285 276 267
    yc          (valid_time) int64 366 367 368 371 372 ... 455 459 464 467 470
    eta         (valid_time, z, dy, dx) float32 dask.array<chunksize=(37, 55, 301, 301), meta=np.ndarray>
    Ja          (valid_time, z, dy, dx) float64 dask.array<chunksize=(37, 55, 301, 301), meta=np.ndarray>
    ...          ...
    divJh       (valid_time, z, dy, dx) float32 dask.array<chunksize=(37, 55, 301, 301), meta=np.ndarray>
    divJe       (valid_time, z, dy, dx) float32 dask.array<chunksize=(37, 55, 301, 301), meta=np.ndarray>
    pres        (valid_time, z, dy, dx) float32 nan nan ... 1.207e+03 1.208e+03
    cimf        (valid_time, dy, dx, z) float32 nan nan nan ... -0.00249 0.0
    sigma       (valid_time, z, dy, dx) float64 nan nan nan ... 0.6139 0.6139
    radius      (valid_time, dy, dx) float32 6.79e+05 6.767e+05 ... 6.745e+05
Attributes:
    GRIB_edition:            1
    GRIB_centre:             eidb
    GRIB_centreDescription:  Dublin
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Dublin
    history:                 2022-03-29T13:13 GRIB to CDM+CF via cfgrib-0.9.9...

In [4]:
dst = xarray.open_mfdataset(files, combine='nested', concat_dim='valid_time')

In [5]:
dtdp = dst.theta.diff('theta') / dst.pres.diff('theta')

In [6]:
%%time
ds = xarray.open_mfdataset(files, combine='nested', concat_dim='valid_time')
ds = ds.drop_vars(['q','tke','height','Ke','dtdp'])
ds = c.crop(ds)
ds = ds.swap_dims(theta='z')
ds['cimf'] = c.calc_cimf(ds)
dvdx = ddxND(ds.v, ds.latitude, ds.longitude)
dudy = ddyND(ds.u, ds.latitude, ds.longitude)
av = (dvdx - dudy + c.fc).transpose(*ds.u.dims)
av = xarray.DataArray(av, coords=ds.u.coords, dims=ds.u.dims, name='eta')
av.attrs = {'long_name':'absolute vorticity', 'units':'s**-1'}
ds['eta'] = av
dp = c.calc_dp(ds)
dth = ds.theta.differentiate('z')
sigma = -1/c.g*dp/dth
ds['dthetadt'] = ds.cimf/sigma
ds = c.convert_wind(ds)
ds['dvdtheta'] = ds.v_tan.differentiate('theta')

08:08:49: crop()
08:09:08: calc_cimf()
08:09:08: masscontinuity(lagrange2d)
08:09:08: latlon2dxdy()
08:09:08: translational_velocity()
08:09:08: calc_dp(ds)
08:09:09: masstendency()
08:09:10: iwind2
08:09:12: FX, FY in masscontinuity()
08:09:30: ddx, ddy in masscontinuity()
08:09:54: integrate_dcimf()
08:10:07: calc_dp(ds)
08:10:12: convert_wind()
08:10:12: toPolar()
08:10:12: distance()
08:10:12: translational_velocity()
08:10:17: transform_vector()
CPU times: user 53 s, sys: 1min 6s, total: 1min 59s
Wall time: 2min 2s


In [7]:
dss = ds[['latitude','longitude','xc','yc','u_rad','v_tan','eta','dthetadt','dvdtheta','pres']]
dp = c.calc_dp(ds)
dth = ds.theta.differentiate('z')
dss['sigma'] = -1/c.g*dp/dth
dss['dthetadt'] = ds.cimf/dss.sigma

08:10:49: calc_dp(ds)


In [8]:
%%time
dsm,_ = c.azimean(dss)

08:10:59: azimean()
08:10:59: toPolar()
08:10:59: distance()
08:11:01: convert_wind()
08:11:01: convert_wind(): using existing u_rad and v_tan
08:11:01: azimean_gufunc()
08:11:35: removing ['dy', 'dx'] from azimuthal mean dataset
CPU times: user 14.7 s, sys: 12 s, total: 26.7 s
Wall time: 35.5 s


In [9]:
dsm['Ja_m'] = dsm.u_rad * dsm.eta
dsm['Jh_m'] = dsm.dthetadt * dsm.dvdtheta

In [10]:
# convert azimuthal means back to original grid
dist, angle = c.toPolar(ds.latitude, ds.longitude, ds.xc, ds.yc)
dist.attrs = {'long_name':'radial distance from centre','units':'m'}
ds['radius'] = dist.compute().astype('float32')
u_rad_m = dsm.u_rad.interp(r=ds.radius)
eta_m = dsm.eta.interp(r=ds.radius)
dthetadt_m = dsm.dthetadt.interp(r=ds.radius)
dvdtheta_m = dsm.dvdtheta.interp(r=ds.radius)

08:11:35: toPolar()
08:11:35: distance()


In [11]:
# calculate eddy fluxes
u_e = ds['u_rad'] - u_rad_m
eta_e = ds['eta'] - eta_m 
dthetadt_e = ds['dthetadt'] - dthetadt_m 
dvdtheta_e = ds['dvdtheta'] - dvdtheta_m
ds['Ja_e'] = u_e * eta_e
ds['Jh_e'] = dthetadt_e * dvdtheta_e

In [12]:
# azimuthal mean of eddy components
dsme,_ = c.azimean(ds[['latitude','longitude','xc','yc','Ja_e','Jh_e']])

08:13:56: azimean()
08:13:56: toPolar()
08:13:56: distance()
08:13:57: convert_wind()
08:13:57: convert_wind(): no input, skipping
08:13:57: azimean_gufunc()
08:14:04: removing ['dy', 'dx'] from azimuthal mean dataset


In [13]:
dsm = xarray.merge((dsm, dsme))
dsm = dsm.sel(valid_time=slice('2017-09-06 00','2017-09-06 12')).mean('valid_time')

In [15]:
dsm.to_netcdf('lg2meaneddy.nc')